<a href="https://colab.research.google.com/github/tiagopessoalima/ED2/blob/main/Semana_12_(ED2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Árvores Binárias de Busca (BST)

## 1. Introdução às Árvores Binárias de Busca

### 1.1 Definição e Propriedades

Uma Árvore Binária de Busca (BST - Binary Search Tree) é um tipo especial de árvore binária que mantém uma ordenação específica entre seus elementos:

- Para qualquer nó na árvore, todos os valores na subárvore esquerda são **menores** que o valor do nó
- Para qualquer nó na árvore, todos os valores na subárvore direita são **maiores** que o valor do nó
- Tanto a subárvore esquerda quanto a direita também são árvores binárias de busca

Esta propriedade de ordenação é o que torna as BSTs tão úteis para operações de busca, inserção e remoção eficientes.

### 1.2 Vantagens das Árvores Binárias de Busca

- **Busca eficiente**: Operações de busca têm complexidade média O(log n), onde n é o número de nós
- **Inserção e remoção eficientes**: Também têm complexidade média O(log n)
- **Manutenção da ordenação**: Os elementos ficam naturalmente ordenados (percurso em-ordem)
- **Flexibilidade**: Podem ser balanceadas para garantir desempenho ótimo

### 1.3 Limitações das Árvores Binárias de Busca

- **Degeneração**: No pior caso (árvore degenerada em uma lista), a complexidade das operações se torna O(n)
- **Sensibilidade à ordem de inserção**: A forma da árvore depende da ordem em que os elementos são inseridos
- **Necessidade de balanceamento**: Para garantir desempenho ótimo, podem ser necessárias técnicas de balanceamento

## 2. Implementação Básica de uma Árvore Binária de Busca

Vamos implementar uma BST em Python, começando com a definição da classe de nó e as operações básicas.

In [ ]:
class No:
    def __init__(self, valor):
        self.valor = valor        # Valor armazenado no nó
        self.esquerda = None      # Referência para o filho esquerdo (valores menores)
        self.direita = None       # Referência para o filho direito (valores maiores)

    def __str__(self):
        return f"Nó({self.valor})"

class ArvoreBinariaBusca:
    def __init__(self):
        self.raiz = None          # Inicialmente, a árvore está vazia

    def esta_vazia(self):
        """Verifica se a árvore está vazia."""
        return self.raiz is None

    def __str__(self):
        if self.esta_vazia():
            return "Árvore vazia"
        return f"BST com raiz {self.raiz.valor}"

# Criando uma BST vazia
bst = ArvoreBinariaBusca()
print(bst)

### 2.1 Visualização de Árvores Binárias de Busca

Antes de implementar as operações, vamos criar funções para visualizar a estrutura da árvore, o que nos ajudará a entender melhor o comportamento das operações.

In [ ]:
def imprimir_arvore(no, nivel=0, prefixo="Raiz: "):
    """Imprime a estrutura da árvore de forma textual."""
    if no is not None:
        espacos = "    " * nivel
        print(f"{espacos}{prefixo}{no.valor}")
        if no.esquerda is not None or no.direita is not None:  # Se tiver pelo menos um filho
            if no.esquerda:
                imprimir_arvore(no.esquerda, nivel + 1, "E── ")
            else:
                print(f"{espacos}    E── None")
            if no.direita:
                imprimir_arvore(no.direita, nivel + 1, "D── ")
            else:
                print(f"{espacos}    D── None")

# Função para visualização gráfica usando matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def calcular_posicoes(no, nivel=0, offset=0, posicoes=None, largura_nivel=None):
    """Calcula as posições dos nós para visualização gráfica."""
    if posicoes is None:
        posicoes = {}
    if largura_nivel is None:
        largura_nivel = {}

    if no is None:
        return posicoes, largura_nivel

    # Inicializa a largura do nível se ainda não existir
    if nivel not in largura_nivel:
        largura_nivel[nivel] = 0

    # Calcula posições para a subárvore esquerda
    posicoes, largura_nivel = calcular_posicoes(no.esquerda, nivel + 1, offset, posicoes, largura_nivel)

    # Posiciona o nó atual
    x = offset + largura_nivel[nivel]
    y = nivel
    posicoes[no] = (x, y)
    largura_nivel[nivel] += 1

    # Calcula posições para a subárvore direita
    posicoes, largura_nivel = calcular_posicoes(no.direita, nivel + 1, offset + largura_nivel[nivel], posicoes, largura_nivel)

    return posicoes, largura_nivel

def visualizar_arvore(raiz, titulo="Visualização da Árvore Binária de Busca"):
    """Visualiza a árvore graficamente usando matplotlib."""
    if raiz is None:
        print("Árvore vazia")
        return

    posicoes, _ = calcular_posicoes(raiz)

    # Configuração do plot
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_axis_off()

    # Desenha os nós e as arestas
    for no, (x, y) in posicoes.items():
        # Desenha o nó
        circle = patches.Circle((x, -y), 0.3, facecolor='lightblue', edgecolor='blue')
        ax.add_patch(circle)
        ax.text(x, -y, str(no.valor), ha='center', va='center')

        # Desenha as arestas para os filhos
        if no.esquerda and no.esquerda in posicoes:
            child_x, child_y = posicoes[no.esquerda]
            ax.plot([x, child_x], [-y, -child_y], 'k-')

        if no.direita and no.direita in posicoes:
            child_x, child_y = posicoes[no.direita]
            ax.plot([x, child_x], [-y, -child_y], 'k-')

    # Ajusta os limites do plot
    xs = [x for x, y in posicoes.values()]
    ys = [-y for x, y in posicoes.values()]
    ax.set_xlim(min(xs) - 1, max(xs) + 1)
    ax.set_ylim(min(ys) - 1, max(ys) + 1)

    plt.title(titulo)
    plt.tight_layout()
    plt.show()

## 3. Operações Básicas em Árvores Binárias de Busca

### 3.1 Inserção

A inserção em uma BST segue a propriedade de ordenação: valores menores vão para a esquerda, valores maiores vão para a direita.

In [ ]:
def inserir(self, valor):
    """Insere um valor na árvore."""
    if self.raiz is None:
        self.raiz = No(valor)
    else:
        self._inserir_recursivo(self.raiz, valor)

def _inserir_recursivo(self, no, valor):
    """Função auxiliar para inserção recursiva."""
    if valor < no.valor:
        if no.esquerda is None:
            no.esquerda = No(valor)
        else:
            self._inserir_recursivo(no.esquerda, valor)
    elif valor > no.valor:  # Ignoramos valores duplicados
        if no.direita is None:
            no.direita = No(valor)
        else:
            self._inserir_recursivo(no.direita, valor)
    # Se valor == no.valor, não fazemos nada (evitamos duplicatas)

# Adicionando os métodos à classe
ArvoreBinariaBusca.inserir = inserir
ArvoreBinariaBusca._inserir_recursivo = _inserir_recursivo

# Testando a inserção
bst = ArvoreBinariaBusca()
valores = [50, 30, 70, 20, 40, 60, 80, 35, 45]

for valor in valores:
    bst.inserir(valor)

print("Árvore após inserções:")
imprimir_arvore(bst.raiz)
visualizar_arvore(bst.raiz)

### 3.2 Busca

A busca em uma BST aproveita a propriedade de ordenação para reduzir o espaço de busca pela metade a cada passo.

In [ ]:
def buscar(self, valor):
    """Busca um valor na árvore e retorna o nó se encontrado, ou None caso contrário."""
    return self._buscar_recursivo(self.raiz, valor)

def _buscar_recursivo(self, no, valor):
    """Função auxiliar para busca recursiva."""
    # Caso base: árvore vazia ou valor encontrado
    if no is None or no.valor == valor:
        return no

    # Caso recursivo: busca na subárvore apropriada
    if valor < no.valor:
        return self._buscar_recursivo(no.esquerda, valor)
    else:  # valor > no.valor
        return self._buscar_recursivo(no.direita, valor)

def contem(self, valor):
    """Verifica se a árvore contém um valor específico."""
    return self.buscar(valor) is not None

# Adicionando os métodos à classe
ArvoreBinariaBusca.buscar = buscar
ArvoreBinariaBusca._buscar_recursivo = _buscar_recursivo
ArvoreBinariaBusca.contem = contem

# Testando a busca
valores_busca = [40, 90, 20]
for valor in valores_busca:
    resultado = bst.contem(valor)
    print(f"A árvore contém o valor {valor}? {resultado}")

### 3.3 Percursos

Os percursos em uma BST são os mesmos de uma árvore binária comum, mas o percurso em-ordem tem uma propriedade especial: ele visita os nós em ordem crescente de valores.

In [ ]:
def pre_ordem(self):
    """Retorna os valores da árvore em pré-ordem (raiz, esquerda, direita)."""
    resultado = []
    self._pre_ordem_recursivo(self.raiz, resultado)
    return resultado

def _pre_ordem_recursivo(self, no, resultado):
    if no is not None:
        resultado.append(no.valor)  # Visita o nó atual
        self._pre_ordem_recursivo(no.esquerda, resultado)  # Visita a subárvore esquerda
        self._pre_ordem_recursivo(no.direita, resultado)  # Visita a subárvore direita

def em_ordem(self):
    """Retorna os valores da árvore em em-ordem (esquerda, raiz, direita)."""
    resultado = []
    self._em_ordem_recursivo(self.raiz, resultado)
    return resultado

def _em_ordem_recursivo(self, no, resultado):
    if no is not None:
        self._em_ordem_recursivo(no.esquerda, resultado)  # Visita a subárvore esquerda
        resultado.append(no.valor)  # Visita o nó atual
        self._em_ordem_recursivo(no.direita, resultado)  # Visita a subárvore direita

def pos_ordem(self):
    """Retorna os valores da árvore em pós-ordem (esquerda, direita, raiz)."""
    resultado = []
    self._pos_ordem_recursivo(self.raiz, resultado)
    return resultado

def _pos_ordem_recursivo(self, no, resultado):
    if no is not None:
        self._pos_ordem_recursivo(no.esquerda, resultado)  # Visita a subárvore esquerda
        self._pos_ordem_recursivo(no.direita, resultado)  # Visita a subárvore direita
        resultado.append(no.valor)  # Visita o nó atual

# Adicionando os métodos à classe
ArvoreBinariaBusca.pre_ordem = pre_ordem
ArvoreBinariaBusca._pre_ordem_recursivo = _pre_ordem_recursivo
ArvoreBinariaBusca.em_ordem = em_ordem
ArvoreBinariaBusca._em_ordem_recursivo = _em_ordem_recursivo
ArvoreBinariaBusca.pos_ordem = pos_ordem
ArvoreBinariaBusca._pos_ordem_recursivo = _pos_ordem_recursivo

# Testando os percursos
print(f"Percurso em pré-ordem: {bst.pre_ordem()}")
print(f"Percurso em em-ordem (ordenado): {bst.em_ordem()}")
print(f"Percurso em pós-ordem: {bst.pos_ordem()}")

### 3.4 Encontrar Valores Mínimo e Máximo

Em uma BST, o valor mínimo está sempre no nó mais à esquerda, e o valor máximo está sempre no nó mais à direita.

In [ ]:
def encontrar_minimo(self):
    """Retorna o valor mínimo na árvore."""
    if self.esta_vazia():
        return None

    no_atual = self.raiz
    while no_atual.esquerda is not None:
        no_atual = no_atual.esquerda

    return no_atual.valor

def encontrar_maximo(self):
    """Retorna o valor máximo na árvore."""
    if self.esta_vazia():
        return None

    no_atual = self.raiz
    while no_atual.direita is not None:
        no_atual = no_atual.direita

    return no_atual.valor

# Adicionando os métodos à classe
ArvoreBinariaBusca.encontrar_minimo = encontrar_minimo
ArvoreBinariaBusca.encontrar_maximo = encontrar_maximo

# Testando os métodos
print(f"Valor mínimo na árvore: {bst.encontrar_minimo()}")
print(f"Valor máximo na árvore: {bst.encontrar_maximo()}")

### 3.5 Remoção

A remoção em uma BST é mais complexa que a inserção e a busca, pois precisamos manter a propriedade de ordenação após remover um nó. Existem três casos a considerar:

1. **Nó folha (sem filhos)**: Simplesmente removemos o nó.
2. **Nó com um filho**: Substituímos o nó pelo seu filho.
3. **Nó com dois filhos**: Substituímos o nó pelo seu sucessor in-order (o menor valor na subárvore direita) ou pelo seu predecessor in-order (o maior valor na subárvore esquerda).

In [ ]:
def remover(self, valor):
    """Remove um valor da árvore."""
    self.raiz = self._remover_recursivo(self.raiz, valor)

def _remover_recursivo(self, no, valor):
    """Função auxiliar para remoção recursiva."""
    # Caso base: árvore vazia
    if no is None:
        return None

    # Busca o nó a ser removido
    if valor < no.valor:
        no.esquerda = self._remover_recursivo(no.esquerda, valor)
    elif valor > no.valor:
        no.direita = self._remover_recursivo(no.direita, valor)
    else:
        # Caso 1: Nó folha (sem filhos)
        if no.esquerda is None and no.direita is None:
            return None

        # Caso 2: Nó com apenas um filho
        elif no.esquerda is None:
            return no.direita
        elif no.direita is None:
            return no.esquerda

        # Caso 3: Nó com dois filhos
        # Encontra o sucessor in-order (menor valor na subárvore direita)
        sucessor = self._encontrar_no_minimo(no.direita)
        no.valor = sucessor.valor
        no.direita = self._remover_recursivo(no.direita, sucessor.valor)

    return no

def _encontrar_no_minimo(self, no):
    """Encontra o nó com o valor mínimo na subárvore."""
    atual = no
    while atual.esquerda is not None:
        atual = atual.esquerda
    return atual

# Adicionando os métodos à classe
ArvoreBinariaBusca.remover = remover
ArvoreBinariaBusca._remover_recursivo = _remover_recursivo
ArvoreBinariaBusca._encontrar_no_minimo = _encontrar_no_minimo

# Testando a remoção
print("\nÁrvore original:")
imprimir_arvore(bst.raiz)

# Caso 1: Removendo um nó folha (45)
bst.remover(45)
print("\nApós remover o nó folha 45:")
imprimir_arvore(bst.raiz)

# Caso 2: Removendo um nó com um filho (40)
bst.remover(40)
print("\nApós remover o nó 40 (que tem um filho):")
imprimir_arvore(bst.raiz)

# Caso 3: Removendo um nó com dois filhos (30)
bst.remover(30)
print("\nApós remover o nó 30 (que tem dois filhos):")
imprimir_arvore(bst.raiz)
visualizar_arvore(bst.raiz, "Árvore após todas as remoções")

## 4. Análise de Complexidade

A eficiência das operações em uma BST depende da altura da árvore. No caso médio (árvore relativamente balanceada), temos:

| Operação | Complexidade de Tempo (Caso Médio) | Complexidade de Tempo (Pior Caso) |
|----------|-----------------------------------|----------------------------------|
| Busca    | O(log n)                          | O(n)                             |
| Inserção | O(log n)                          | O(n)                             |
| Remoção  | O(log n)                          | O(n)                             |

O pior caso ocorre quando a árvore está completamente desbalanceada (degenerada em uma lista ligada), o que pode acontecer se os elementos forem inseridos em ordem crescente ou decrescente.

### 4.1 Demonstração do Pior Caso

Vamos demonstrar como a ordem de inserção afeta a estrutura da árvore e, consequentemente, a eficiência das operações.

In [ ]:
# Criando uma BST com elementos em ordem crescente (pior caso)
bst_pior_caso = ArvoreBinariaBusca()
for i in range(1, 8):
    bst_pior_caso.inserir(i)

print("BST com inserções em ordem crescente (pior caso):")
imprimir_arvore(bst_pior_caso.raiz)
visualizar_arvore(bst_pior_caso.raiz, "BST Degenerada (Pior Caso)")

# Criando uma BST com elementos em ordem balanceada (caso médio/melhor)
bst_balanceada = ArvoreBinariaBusca()
valores_balanceados = [4, 2, 6, 1, 3, 5, 7]  # Inserção que naturalmente balanceia
for valor in valores_balanceados:
    bst_balanceada.inserir(valor)

print("\nBST com inserções balanceadas (caso médio/melhor):")
imprimir_arvore(bst_balanceada.raiz)
visualizar_arvore(bst_balanceada.raiz, "BST Balanceada (Caso Médio/Melhor)")

# Calculando a altura das árvores
def altura(no):
    if no is None:
        return -1
    return 1 + max(altura(no.esquerda), altura(no.direita))

print(f"Altura da BST degenerada: {altura(bst_pior_caso.raiz)}")
print(f"Altura da BST balanceada: {altura(bst_balanceada.raiz)}")

## 5. Algoritmos Avançados em BST

### 5.1 Verificação de BST

Um algoritmo para verificar se uma árvore binária é uma BST válida:

In [ ]:
def eh_bst_valida(raiz, minimo=float('-inf'), maximo=float('inf')):
    """Verifica se uma árvore binária é uma BST válida."""
    if raiz is None:
        return True

    # Verifica se o valor do nó está dentro dos limites permitidos
    if raiz.valor <= minimo or raiz.valor >= maximo:
        return False

    # Verifica recursivamente as subárvores
    return (eh_bst_valida(raiz.esquerda, minimo, raiz.valor) and
            eh_bst_valida(raiz.direita, raiz.valor, maximo))

# Testando com uma BST válida
print(f"A árvore balanceada é uma BST válida? {eh_bst_valida(bst_balanceada.raiz)}")

# Criando uma árvore binária que não é BST
nao_bst = No(10)
nao_bst.esquerda = No(5)
nao_bst.direita = No(15)
nao_bst.esquerda.direita = No(20)  # Viola a propriedade BST (20 > 10)

print(f"A árvore não-BST é uma BST válida? {eh_bst_valida(nao_bst)}")

### 5.2 Encontrar o Predecessor e Sucessor

O predecessor de um nó é o nó com o maior valor menor que o valor do nó atual.
O sucessor de um nó é o nó com o menor valor maior que o valor do nó atual.

In [ ]:
def encontrar_sucessor(self, valor):
    """Encontra o sucessor de um valor na árvore."""
    no = self.buscar(valor)
    if no is None:
        return None

    # Caso 1: O nó tem subárvore direita
    if no.direita is not None:
        return self._encontrar_minimo_valor(no.direita)

    # Caso 2: O nó não tem subárvore direita
    # Precisamos encontrar o ancestral mais próximo cujo filho esquerdo é um ancestral do nó
    sucessor = None
    ancestral = self.raiz

    while ancestral != no:
        if no.valor < ancestral.valor:
            sucessor = ancestral
            ancestral = ancestral.esquerda
        else:
            ancestral = ancestral.direita

    return sucessor.valor if sucessor else None

def encontrar_predecessor(self, valor):
    """Encontra o predecessor de um valor na árvore."""
    no = self.buscar(valor)
    if no is None:
        return None

    # Caso 1: O nó tem subárvore esquerda
    if no.esquerda is not None:
        return self._encontrar_maximo_valor(no.esquerda)

    # Caso 2: O nó não tem subárvore esquerda
    # Precisamos encontrar o ancestral mais próximo cujo filho direito é um ancestral do nó
    predecessor = None
    ancestral = self.raiz

    while ancestral != no:
        if no.valor > ancestral.valor:
            predecessor = ancestral
            ancestral = ancestral.direita
        else:
            ancestral = ancestral.esquerda

    return predecessor.valor if predecessor else None

def _encontrar_minimo_valor(self, no):
    """Encontra o valor mínimo na subárvore."""
    atual = no
    while atual.esquerda is not None:
        atual = atual.esquerda
    return atual.valor

def _encontrar_maximo_valor(self, no):
    """Encontra o valor máximo na subárvore."""
    atual = no
    while atual.direita is not None:
        atual = atual.direita
    return atual.valor

# Adicionando os métodos à classe
ArvoreBinariaBusca.encontrar_sucessor = encontrar_sucessor
ArvoreBinariaBusca.encontrar_predecessor = encontrar_predecessor
ArvoreBinariaBusca._encontrar_minimo_valor = _encontrar_minimo_valor
ArvoreBinariaBusca._encontrar_maximo_valor = _encontrar_maximo_valor

# Testando com a árvore balanceada
print("Árvore balanceada:")
imprimir_arvore(bst_balanceada.raiz)

for valor in [1, 2, 4, 7]:
    predecessor = bst_balanceada.encontrar_predecessor(valor)
    sucessor = bst_balanceada.encontrar_sucessor(valor)
    print(f"Valor {valor}: Predecessor = {predecessor}, Sucessor = {sucessor}")

### 5.3 Encontrar o k-ésimo Menor Elemento

Podemos usar o percurso em-ordem para encontrar o k-ésimo menor elemento na BST.

In [ ]:
def encontrar_k_esimo_menor(self, k):
    """Encontra o k-ésimo menor elemento na árvore (k começa em 1)."""
    if k <= 0 or self.esta_vazia():
        return None

    # Obtém todos os elementos em ordem crescente
    elementos = self.em_ordem()

    # Verifica se k é válido
    if k > len(elementos):
        return None

    # Retorna o k-ésimo elemento (ajustando o índice baseado em 0)
    return elementos[k-1]

# Adicionando o método à classe
ArvoreBinariaBusca.encontrar_k_esimo_menor = encontrar_k_esimo_menor

# Testando com a árvore balanceada
for k in range(1, 8):
    elemento = bst_balanceada.encontrar_k_esimo_menor(k)
    print(f"{k}º menor elemento: {elemento}")

### 5.4 Menor Ancestral Comum (LCA)

O Menor Ancestral Comum (LCA - Lowest Common Ancestor) de dois nós em uma BST é o nó mais profundo que tem ambos os nós como descendentes.

In [ ]:
def encontrar_lca(self, valor1, valor2):
    """Encontra o menor ancestral comum de dois valores na árvore."""
    return self._encontrar_lca_recursivo(self.raiz, valor1, valor2)

def _encontrar_lca_recursivo(self, no, valor1, valor2):
    """Função auxiliar para encontrar o LCA recursivamente."""
    if no is None:
        return None

    # Se ambos os valores são menores que o nó atual, o LCA está na subárvore esquerda
    if valor1 < no.valor and valor2 < no.valor:
        return self._encontrar_lca_recursivo(no.esquerda, valor1, valor2)

    # Se ambos os valores são maiores que o nó atual, o LCA está na subárvore direita
    if valor1 > no.valor and valor2 > no.valor:
        return self._encontrar_lca_recursivo(no.direita, valor1, valor2)

    # Se um valor é menor e o outro é maior (ou igual), o nó atual é o LCA
    return no.valor

# Adicionando os métodos à classe
ArvoreBinariaBusca.encontrar_lca = encontrar_lca
ArvoreBinariaBusca._encontrar_lca_recursivo = _encontrar_lca_recursivo

# Testando com a árvore balanceada
pares = [(1, 3), (5, 7), (1, 7), (2, 6)]
for valor1, valor2 in pares:
    lca = bst_balanceada.encontrar_lca(valor1, valor2)
    print(f"LCA de {valor1} e {valor2}: {lca}")

## 6. Balanceamento de Árvores Binárias de Busca

Como vimos, a eficiência das operações em uma BST depende da sua altura. Para garantir que a altura seja sempre O(log n), precisamos manter a árvore balanceada.

Existem várias técnicas de balanceamento, como:

1. **Árvores AVL**: Mantêm o fator de balanceamento (diferença de altura entre subárvores esquerda e direita) de cada nó entre -1 e 1.
2. **Árvores Rubro-Negras**: Usam cores (vermelho e preto) e um conjunto de regras para manter o balanceamento.
3. **Árvores B e B+**: Generalizam a ideia de BST para permitir mais de dois filhos por nó.

Vamos implementar uma versão simplificada de balanceamento, reconstruindo a árvore a partir de um array ordenado:

In [ ]:
def balancear(self):
    """Balanceia a árvore reconstruindo-a a partir de um array ordenado."""
    # Obtém todos os valores em ordem crescente
    valores = self.em_ordem()

    # Limpa a árvore atual
    self.raiz = None

    # Reconstrói a árvore de forma balanceada
    self._construir_balanceada(valores, 0, len(valores) - 1)

def _construir_balanceada(self, valores, inicio, fim):
    """Constrói uma BST balanceada a partir de um array ordenado."""
    if inicio > fim:
        return

    # Encontra o meio do array
    meio = (inicio + fim) // 2

    # Insere o elemento do meio como raiz
    self.inserir(valores[meio])

    # Constrói recursivamente as subárvores esquerda e direita
    self._construir_balanceada(valores, inicio, meio - 1)
    self._construir_balanceada(valores, meio + 1, fim)

# Adicionando os métodos à classe
ArvoreBinariaBusca.balancear = balancear
ArvoreBinariaBusca._construir_balanceada = _construir_balanceada

# Testando o balanceamento com a árvore degenerada
print("BST degenerada antes do balanceamento:")
imprimir_arvore(bst_pior_caso.raiz)
visualizar_arvore(bst_pior_caso.raiz, "Antes do Balanceamento")

# Balanceando a árvore
bst_pior_caso.balancear()

print("\nBST após o balanceamento:")
imprimir_arvore(bst_pior_caso.raiz)
visualizar_arvore(bst_pior_caso.raiz, "Após o Balanceamento")

# Comparando as alturas
print(f"Altura antes do balanceamento: {altura(bst_pior_caso.raiz)}")

## 7. Aplicações Práticas de Árvores Binárias de Busca

As BSTs são amplamente utilizadas em diversas aplicações, como:

1. **Implementação de conjuntos e mapas**: Estruturas como `TreeSet` e `TreeMap` em Java são implementadas usando BSTs balanceadas.
2. **Bancos de dados**: Índices em bancos de dados frequentemente usam variações de BSTs.
3. **Sistemas de arquivos**: Muitos sistemas de arquivos usam BSTs para organizar diretórios e arquivos.
4. **Compressão de dados**: Árvores de Huffman (uma variação de BST) são usadas em algoritmos de compressão.
5. **Roteamento de rede**: Tabelas de roteamento podem ser implementadas usando BSTs.

### 7.1 Exemplo: Implementação de um Dicionário

In [ ]:
class NoDicionario:
    def __init__(self, chave, valor):
        self.chave = chave          # Chave para ordenação
        self.valor = valor          # Valor associado à chave
        self.esquerda = None        # Referência para o filho esquerdo
        self.direita = None         # Referência para o filho direito

    def __str__(self):
        return f"({self.chave}: {self.valor})"

class Dicionario:
    def __init__(self):
        self.raiz = None

    def inserir(self, chave, valor):
        """Insere ou atualiza um par chave-valor no dicionário."""
        self.raiz = self._inserir_recursivo(self.raiz, chave, valor)

    def _inserir_recursivo(self, no, chave, valor):
        if no is None:
            return NoDicionario(chave, valor)

        if chave < no.chave:
            no.esquerda = self._inserir_recursivo(no.esquerda, chave, valor)
        elif chave > no.chave:
            no.direita = self._inserir_recursivo(no.direita, chave, valor)
        else:  # chave == no.chave
            no.valor = valor  # Atualiza o valor se a chave já existe

        return no

    def buscar(self, chave):
        """Busca um valor pela chave."""
        no = self._buscar_recursivo(self.raiz, chave)
        return no.valor if no else None

    def _buscar_recursivo(self, no, chave):
        if no is None or no.chave == chave:
            return no

        if chave < no.chave:
            return self._buscar_recursivo(no.esquerda, chave)
        else:
            return self._buscar_recursivo(no.direita, chave)

    def remover(self, chave):
        """Remove um par chave-valor do dicionário."""
        self.raiz = self._remover_recursivo(self.raiz, chave)

    def _remover_recursivo(self, no, chave):
        if no is None:
            return None

        if chave < no.chave:
            no.esquerda = self._remover_recursivo(no.esquerda, chave)
        elif chave > no.chave:
            no.direita = self._remover_recursivo(no.direita, chave)
        else:
            # Caso 1 e 2: Nó sem filhos ou com apenas um filho
            if no.esquerda is None:
                return no.direita
            elif no.direita is None:
                return no.esquerda

            # Caso 3: Nó com dois filhos
            # Encontra o sucessor in-order
            sucessor = self._encontrar_minimo(no.direita)
            no.chave = sucessor.chave
            no.valor = sucessor.valor
            no.direita = self._remover_recursivo(no.direita, sucessor.chave)

        return no

    def _encontrar_minimo(self, no):
        atual = no
        while atual.esquerda is not None:
            atual = atual.esquerda
        return atual

    def listar_chaves(self):
        """Retorna todas as chaves em ordem crescente."""
        chaves = []
        self._em_ordem(self.raiz, chaves)
        return chaves

    def _em_ordem(self, no, resultado):
        if no is not None:
            self._em_ordem(no.esquerda, resultado)
            resultado.append(no.chave)
            self._em_ordem(no.direita, resultado)

    def imprimir(self):
        """Imprime o dicionário em formato de árvore."""
        self._imprimir_recursivo(self.raiz)

    def _imprimir_recursivo(self, no, nivel=0, prefixo="Raiz: "):
        if no is not None:
            espacos = "    " * nivel
            print(f"{espacos}{prefixo}{no}")
            if no.esquerda is not None or no.direita is not None:
                if no.esquerda:
                    self._imprimir_recursivo(no.esquerda, nivel + 1, "E── ")
                else:
                    print(f"{espacos}    E── None")
                if no.direita:
                    self._imprimir_recursivo(no.direita, nivel + 1, "D── ")
                else:
                    print(f"{espacos}    D── None")

# Testando o dicionário
dicionario = Dicionario()

# Inserindo pares chave-valor
dicionario.inserir("maçã", "apple")
dicionario.inserir("banana", "banana")
dicionario.inserir("laranja", "orange")
dicionario.inserir("uva", "grape")
dicionario.inserir("morango", "strawberry")

print("Dicionário:")
dicionario.imprimir()

# Buscando valores
print(f"\nTraduções:")
for chave in ["maçã", "banana", "pêra"]:
    valor = dicionario.buscar(chave)
    if valor:
        print(f"{chave} -> {valor}")
    else:
        print(f"{chave} não encontrada")

# Atualizando um valor
dicionario.inserir("banana", "yellow fruit")
print(f"\nBanana após atualização: {dicionario.buscar('banana')}")

# Removendo um item
dicionario.remover("laranja")
print("\nDicionário após remover 'laranja':")
dicionario.imprimir()

# Listando todas as chaves
print(f"\nTodas as chaves: {dicionario.listar_chaves()}")

### 7.2 Exemplo: Contagem de Frequência de Palavras

In [ ]:
def contar_frequencia_palavras(texto):
    """Conta a frequência de cada palavra em um texto usando um dicionário baseado em BST."""
    # Pré-processamento do texto
    texto = texto.lower()  # Converte para minúsculas
    # Remove pontuação e divide em palavras
    palavras = ''.join(c if c.isalnum() or c.isspace() else ' ' for c in texto).split()

    # Cria um dicionário para contar as frequências
    contador = Dicionario()

    # Conta a frequência de cada palavra
    for palavra in palavras:
        freq = contador.buscar(palavra)
        if freq is None:
            contador.inserir(palavra, 1)
        else:
            contador.inserir(palavra, freq + 1)

    return contador

# Testando a contagem de frequência
texto = """Árvores binárias de busca são estruturas de dados eficientes para busca, inserção e remoção.
As árvores binárias de busca mantêm seus elementos ordenados, o que facilita operações como busca binária.
Quando uma árvore binária de busca está balanceada, suas operações têm complexidade logarítmica."""

contador = contar_frequencia_palavras(texto)

# Obtendo as palavras em ordem alfabética
palavras = contador.listar_chaves()

print("Frequência de palavras:")
for palavra in palavras:
    freq = contador.buscar(palavra)
    print(f"{palavra}: {freq}")

# Encontrando as palavras mais frequentes
palavras_mais_frequentes = []
max_freq = 0

for palavra in palavras:
    freq = contador.buscar(palavra)
    if freq > max_freq:
        palavras_mais_frequentes = [palavra]
        max_freq = freq
    elif freq == max_freq:
        palavras_mais_frequentes.append(palavra)

print(f"\nPalavras mais frequentes ({max_freq} ocorrências): {', '.join(palavras_mais_frequentes)}")

## 8. Comparação com Outras Estruturas de Dados

Vamos comparar as BSTs com outras estruturas de dados comuns:

| Estrutura de Dados | Busca | Inserção | Remoção | Ordenação | Espaço |
|-------------------|-------|----------|---------|-----------|--------|
| Array não ordenado | O(n) | O(1) | O(n) | O(n log n) | O(n) |
| Array ordenado | O(log n) | O(n) | O(n) | O(1) | O(n) |
| Lista ligada | O(n) | O(1) | O(n) | O(n log n) | O(n) |
| BST (caso médio) | O(log n) | O(log n) | O(log n) | O(n) | O(n) |
| BST (pior caso) | O(n) | O(n) | O(n) | O(n) | O(n) |
| BST balanceada | O(log n) | O(log n) | O(log n) | O(n) | O(n) |
| Tabela hash | O(1)* | O(1)* | O(1)* | O(n log n) | O(n) |

\* Complexidade média, assumindo uma boa função hash e fator de carga adequado.

### 8.1 Quando Usar BSTs

- Quando precisamos manter os dados ordenados
- Quando precisamos de operações eficientes de busca, inserção e remoção
- Quando precisamos encontrar o elemento mínimo/máximo rapidamente
- Quando precisamos encontrar o predecessor/sucessor de um elemento
- Quando o espaço de memória é uma preocupação (BSTs são mais eficientes em espaço que tabelas hash)

### 8.2 Quando Evitar BSTs

- Quando a ordem de inserção pode levar a uma árvore desbalanceada (nesse caso, use uma BST balanceada)
- Quando o tempo de busca constante é crítico (nesse caso, considere tabelas hash)
- Quando os dados são muito grandes e não cabem na memória principal (considere estruturas de dados em disco)

## 9. Exercícios Práticos

### Exercício 1: Implementar uma função para verificar se duas BSTs são idênticas

In [ ]:
def sao_identicas(raiz1, raiz2):
    """Verifica se duas árvores binárias são idênticas."""
    # Se ambas são nulas, são idênticas
    if raiz1 is None and raiz2 is None:
        return True

    # Se uma é nula e a outra não, não são idênticas
    if raiz1 is None or raiz2 is None:
        return False

    # Verifica se os valores são iguais e se as subárvores são idênticas
    return (raiz1.valor == raiz2.valor and
            sao_identicas(raiz1.esquerda, raiz2.esquerda) and
            sao_identicas(raiz1.direita, raiz2.direita))

# Criando duas árvores para teste
bst1 = ArvoreBinariaBusca()
bst2 = ArvoreBinariaBusca()

for valor in [5, 3, 7, 2, 4, 6, 8]:
    bst1.inserir(valor)
    bst2.inserir(valor)

# Criando uma terceira árvore com estrutura diferente
bst3 = ArvoreBinariaBusca()
for valor in [5, 3, 7, 2, 4, 8, 6]:  # Ordem diferente de inserção
    bst3.inserir(valor)

print(f"BST1 e BST2 são idênticas? {sao_identicas(bst1.raiz, bst2.raiz)}")
print(f"BST1 e BST3 são idênticas? {sao_identicas(bst1.raiz, bst3.raiz)}")

### Exercício 2: Implementar uma função para encontrar a soma de todos os valores em uma BST

In [ ]:
def soma_valores(raiz):
    """Calcula a soma de todos os valores na árvore."""
    if raiz is None:
        return 0

    return raiz.valor + soma_valores(raiz.esquerda) + soma_valores(raiz.direita)

# Testando a função
print(f"Soma dos valores em BST1: {soma_valores(bst1.raiz)}")

# Criando uma BST com valores negativos
bst_neg = ArvoreBinariaBusca()
for valor in [10, -5, 15, -10, 0, 12, 20]:
    bst_neg.inserir(valor)

print(f"Soma dos valores em BST com negativos: {soma_valores(bst_neg.raiz)}")

### Exercício 3: Implementar uma função para converter uma BST em uma lista duplamente encadeada, mantendo a ordem

In [ ]:
class NoLista:
    def __init__(self, valor):
        self.valor = valor
        self.anterior = None
        self.proximo = None

def bst_para_lista_duplamente_encadeada(raiz):
    """Converte uma BST em uma lista duplamente encadeada, mantendo a ordem."""
    if raiz is None:
        return None, None  # Retorna cabeça e cauda da lista

    # Converte a subárvore esquerda
    cabeca_esq, cauda_esq = bst_para_lista_duplamente_encadeada(raiz.esquerda)

    # Cria um nó para o valor atual
    no_atual = NoLista(raiz.valor)

    # Conecta com a lista da subárvore esquerda
    if cauda_esq is not None:
        cauda_esq.proximo = no_atual
        no_atual.anterior = cauda_esq

    # Converte a subárvore direita
    cabeca_dir, cauda_dir = bst_para_lista_duplamente_encadeada(raiz.direita)

    # Conecta com a lista da subárvore direita
    if cabeca_dir is not None:
        no_atual.proximo = cabeca_dir
        cabeca_dir.anterior = no_atual

    # Determina a cabeça e a cauda da lista resultante
    cabeca = cabeca_esq if cabeca_esq is not None else no_atual
    cauda = cauda_dir if cauda_dir is not None else no_atual

    return cabeca, cauda

def imprimir_lista(cabeca):
    """Imprime a lista duplamente encadeada."""
    atual = cabeca
    resultado = []
    while atual is not None:
        resultado.append(str(atual.valor))
        atual = atual.proximo
    return " <-> ".join(resultado)

# Testando a conversão
bst_teste = ArvoreBinariaBusca()
for valor in [5, 3, 7, 2, 4, 6, 8]:
    bst_teste.inserir(valor)

print("BST original:")
imprimir_arvore(bst_teste.raiz)

cabeca, _ = bst_para_lista_duplamente_encadeada(bst_teste.raiz)
print(f"\nLista duplamente encadeada: {imprimir_lista(cabeca)}")

## 10. Conclusão

Neste notebook, exploramos em profundidade as Árvores Binárias de Busca (BST), incluindo:

- Definição e propriedades das BSTs
- Implementação básica e operações fundamentais (inserção, busca, remoção)
- Percursos e propriedades especiais das BSTs
- Algoritmos avançados (verificação de BST, predecessor/sucessor, k-ésimo menor elemento, LCA)
- Balanceamento de BSTs
- Aplicações práticas (dicionário, contagem de frequência de palavras)
- Comparação com outras estruturas de dados
- Exercícios práticos para fixação dos conceitos

As Árvores Binárias de Busca são estruturas de dados fundamentais que combinam a flexibilidade das listas ligadas com a eficiência de busca dos arrays ordenados. Quando balanceadas, oferecem operações eficientes de busca, inserção e remoção, mantendo os dados naturalmente ordenados.

### Próximos Passos

Para aprofundar seus conhecimentos em BSTs, considere estudar:

1. Árvores AVL e suas rotações para balanceamento automático
2. Árvores Rubro-Negras e suas regras de balanceamento
3. Árvores B e B+ para armazenamento em disco
4. Árvores de Segmentos (Segment Trees) para consultas de intervalo
5. Árvores de Fenwick (Binary Indexed Trees) para somas cumulativas
6. Implementações de BSTs em bibliotecas padrão de linguagens de programação